![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/healthcare-nlp/08.1.Text_Classification_with_DocumentMLClassifier.ipynb)

# Text Classification with DocumentMLClassifier

## Colab Setup

In [1]:
import json
import os

from google.colab import files

if 'spark_jsl.json' not in os.listdir():
  license_keys = files.upload()
  os.rename(list(license_keys.keys())[0], 'spark_jsl.json')

with open('spark_jsl.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)
os.environ.update(license_keys)

In [ ]:
# Installing pyspark and spark-nlp
%pip install --upgrade -q pyspark==3.2.1 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
%pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

from pyspark.ml import Pipeline,PipelineModel
from pyspark.sql import SparkSession

from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp_jsl.base import *
from sparknlp.base import *
import sparknlp_jsl
import sparknlp

import warnings
warnings.filterwarnings('ignore')

params = {"spark.driver.memory":"12G", 
          "spark.kryoserializer.buffer.max":"2000M", 
          "spark.driver.maxResultSize":"2000M"} 

print ("Spark NLP Version :", sparknlp.version())
print ("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

spark

## Load ADE Classification Dataset

In [4]:
#downloading sample datasets
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/ADE_Corpus_V2/ADE-NEG.txt
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/ADE_Corpus_V2/DRUG-AE.rel

**ADE-Negative Dataset**

In [5]:
df_neg= pd.read_csv("ADE-NEG.txt", header=None, delimiter="\t", names=["col1"])
df_neg['text'] =  df_neg.col1.str.split('NEG').str[1]
df_neg["category"] = "neg"
df_neg= df_neg[["text", "category"]]
df_neg.head()

,text,category
0,Clioquinol intoxication occurring in the trea...,neg
1,"""Retinoic acid syndrome"" was prevented with s...",neg
2,BACKGROUND: External beam radiation therapy o...,neg
3,"Although the enuresis ceased, she developed t...",neg
4,A 42-year-old woman had uneventful bilateral ...,neg


**ADE-Positive Dataset**

In [6]:
df_pos= pd.read_csv("DRUG-AE.rel", header=None, delimiter="|")
df_pos["category"]= "pos"
df_pos.rename(columns={1: "text"}, inplace=True)
df_pos= df_pos[["text", "category"]]
df_pos.head()

,text,category
0,Intravenous azithromycin-induced ototoxicity.,pos
1,"Immobilization, while Paget's bone disease was...",pos
2,Unaccountable severe hypercalcemia in a patien...,pos
3,METHODS: We report two cases of pseudoporphyri...,pos
4,METHODS: We report two cases of pseudoporphyri...,pos


**Merging positive and negative datasets**

In [19]:
ade_df= pd.concat([df_neg, df_pos]).sample(frac=1) #merge and shuffle the data
ade_df.head()

,text,category
15895,It has relatively weak effects on norepinephr...,neg
4329,The pharmacology and toxicology of chloral hyd...,pos
15083,This study provides a review of the medical l...,neg
1180,We describe a case of SCC in vitiligo lesion ...,neg
4110,We describe six patients who developed tardive...,pos


In [20]:
ade_df["category"].value_counts()

neg    16695
pos     6821
Name: category, dtype: int64

In [21]:
ade_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23516 entries, 15895 to 414
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      23516 non-null  object
 1   category  23516 non-null  object
dtypes: object(2)
memory usage: 551.2+ KB


In [22]:
spark_df = spark.createDataFrame(ade_df)
spark_df.head(3)

[Row(text=' It has relatively weak effects on norepinephrine and serotonin reuptake but possesses significant dopamine agonist and reuptake blocking effects.', category='neg'),
 Row(text='The pharmacology and toxicology of chloral hydrate are discussed with particular reference to the cardiac arrhythmias that are seen with overdosage.', category='pos'),
 Row(text=' This study provides a review of the medical literature and presents the intralesional injection technique of 0.1% bleomycin into the wart tissue itself.', category='neg')]

In [23]:
spark_df.groupBy("category").count().show()
spark_df.printSchema()

+--------+-----+
|category|count|
+--------+-----+
|     pos| 6821|
|     neg|16695|
+--------+-----+

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)



**Get train & test set**

In [24]:
train_data, test_data = spark_df.randomSplit([0.8, 0.2], seed = 100)

print("Train Dataset Count: " + str(train_data.count()))
print("Test Dataset Count: " + str(test_data.count()))

Train Dataset Count: 18855
Test Dataset Count: 4661


## DocumentMLClassifier with Linear SVM

The below pipeline uses the SVM classifier of `DocumentMLClassifier` with setting `setClassificationModelClass("svm")`.

In [25]:
document = DocumentAssembler().setInputCol("text").setOutputCol("document")

token = Tokenizer().setInputCols("document").setOutputCol("token")

stemmer = Stemmer() \
    .setInputCols(["token"]) \
    .setOutputCol("stem")

classifier_svm = DocumentMLClassifierApproach() \
    .setInputCols("stem") \
    .setLabelCol("category") \
    .setOutputCol("prediction") \
    .setClassificationModelClass("svm")

pipeline = Pipeline(stages=[document, token, stemmer, classifier_svm])

In [26]:
svm_model = pipeline.fit(train_data)
result_svm = svm_model.transform(test_data).cache()
result_svm.show()

+--------------------+--------+--------------------+--------------------+--------------------+--------------------+
|                text|category|            document|               token|                stem|          prediction|
+--------------------+--------+--------------------+--------------------+--------------------+--------------------+
| A 66-year-old wo...|     neg|[{document, 0, 14...|[{token, 1, 1, A,...|[{token, 1, 1, a,...|[{category, 1, 14...|
| A 73-year-old wo...|     neg|[{document, 0, 11...|[{token, 1, 1, A,...|[{token, 1, 1, a,...|[{category, 1, 11...|
| Further findings...|     neg|[{document, 0, 95...|[{token, 1, 7, Fu...|[{token, 1, 7, fu...|[{category, 1, 95...|
| RMT and AMT were...|     neg|[{document, 0, 11...|[{token, 1, 3, RM...|[{token, 1, 3, rm...|[{category, 1, 11...|
|After stopping in...|     pos|[{document, 0, 70...|[{token, 0, 4, Af...|[{token, 0, 4, af...|[{category, 0, 70...|
|BACKGROUND: Inter...|     pos|[{document, 0, 19...|[{token, 0, 9, BA...

In [27]:
result_svm.printSchema()

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- document: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- token: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- va

In [ ]:
result_svm.select("text","prediction").show(4, truncate=100)

+----------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------+
|                                                                                                text|                                                                      prediction|
+----------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------+
| A 66-year-old woman was diagnosed as having pulmonary adenocarcinoma with malignant pleural effu...|[{category, 1, 148, neg, {sentence -> 0, chunk -> 0, confidence -> 0.6530}, []}]|
| A 73-year-old woman with rheumatoid arthritis had been treated with weekly low-dose methotrexate...|[{category, 1, 115, pos, {sentence -> 0, chunk -> 0, confidence -> 0.9002}, []}]|
|     Further findings included bilateral four-finger hands with hypoplastic thu

In [29]:
result_svm_df = result_svm.select('category','prediction.result').toPandas()
result_svm_df['result'] = result_svm_df.result.apply(lambda x : x[0])

print (classification_report(result_svm_df['category'], result_svm_df['result']))

              precision    recall  f1-score   support

         neg       0.91      0.90      0.91      3317
         pos       0.77      0.77      0.77      1344

    accuracy                           0.87      4661
   macro avg       0.84      0.84      0.84      4661
weighted avg       0.87      0.87      0.87      4661



## DocumentMLClassifier with Logistic Regression

In [30]:
classifier_logreg = DocumentMLClassifierApproach() \
    .setInputCols("token") \
    .setLabelCol("category") \
    .setOutputCol("prediction") \
    .setClassificationModelClass("logreg")\

pipeline = Pipeline(stages=[document, token, classifier_logreg])
    
logreg_model = pipeline.fit(train_data)
result_logreg = logreg_model.transform(test_data).cache()

result_logreg_df = result_logreg.select('category','prediction.result').toPandas()
result_logreg_df['result'] = result_logreg_df.result.apply(lambda x : x[0])

print (classification_report(result_logreg_df['category'], result_logreg_df['result']))

              precision    recall  f1-score   support

         neg       0.90      0.92      0.91      3317
         pos       0.79      0.76      0.77      1344

    accuracy                           0.87      4661
   macro avg       0.85      0.84      0.84      4661
weighted avg       0.87      0.87      0.87      4661



## Playing with Some Parameters

In [31]:
classifier_logreg.extractParamMap()

{Param(parent='DocumentMLClassifierApproach_9f3f7a80b61c', name='lazyAnnotator', doc='Whether this AnnotatorModel acts as lazy in RecursivePipelines'): False,
 Param(parent='DocumentMLClassifierApproach_9f3f7a80b61c', name='labelCol', doc='column with the value result we are trying to predict.'): 'category',
 Param(parent='DocumentMLClassifierApproach_9f3f7a80b61c', name='maxIter', doc='maximum number of iterations.'): 10,
 Param(parent='DocumentMLClassifierApproach_9f3f7a80b61c', name='tol', doc='convergence tolerance after each iteration.'): 1e-06,
 Param(parent='DocumentMLClassifierApproach_9f3f7a80b61c', name='fitIntercept', doc='whether to fit an intercept term, default is true.'): True,
 Param(parent='DocumentMLClassifierApproach_9f3f7a80b61c', name='vectorizationModelPath', doc='specify the vectorization model if it has been already trained.'): '',
 Param(parent='DocumentMLClassifierApproach_9f3f7a80b61c', name='classificationModelPath', doc='specify the classification model if 

**Change `maxIter` parameter with `setMaxIter()`**

In [32]:
classifier_logreg = DocumentMLClassifierApproach() \
    .setInputCols("token") \
    .setLabelCol("category") \
    .setOutputCol("prediction") \
    .setClassificationModelClass("logreg")\
    .setMaxIter(5)

pipeline = Pipeline(stages=[document, token, classifier_logreg])
    
result_logreg = pipeline.fit(train_data).transform(test_data).cache()

result_logreg_df = result_logreg.select('category','prediction.result').toPandas()
result_logreg_df['result'] = result_logreg_df.result.apply(lambda x : x[0])

print (classification_report(result_logreg_df['category'], result_logreg_df['result']))

              precision    recall  f1-score   support

         neg       0.89      0.95      0.92      3317
         pos       0.86      0.71      0.78      1344

    accuracy                           0.88      4661
   macro avg       0.87      0.83      0.85      4661
weighted avg       0.88      0.88      0.88      4661



**Change `tol` (tolarance) parameter with `setTol()`**

In [33]:
classifier_logreg = DocumentMLClassifierApproach() \
    .setInputCols("token") \
    .setLabelCol("category") \
    .setOutputCol("prediction") \
    .setClassificationModelClass("logreg")\
    .setTol(1e-3)

pipeline = Pipeline(stages=[document, token, classifier_logreg])
    
result_logreg = pipeline.fit(train_data).transform(test_data).cache()

result_logreg_df = result_logreg.select('category','prediction.result').toPandas()
result_logreg_df['result'] = result_logreg_df.result.apply(lambda x : x[0])

print (classification_report(result_logreg_df['category'], result_logreg_df['result']))

              precision    recall  f1-score   support

         neg       0.90      0.92      0.91      3317
         pos       0.79      0.76      0.77      1344

    accuracy                           0.87      4661
   macro avg       0.85      0.84      0.84      4661
weighted avg       0.87      0.87      0.87      4661



**Change `maxTokenNgram` parameter with `setMaxTokenNgram()`**

In [34]:
classifier_logreg = DocumentMLClassifierApproach() \
    .setInputCols("token") \
    .setLabelCol("category") \
    .setOutputCol("prediction") \
    .setClassificationModelClass("logreg")\
    .setMaxTokenNgram(2)

pipeline = Pipeline(stages=[document, token, classifier_logreg])
    
result_logreg = pipeline.fit(train_data).transform(test_data).cache()

result_logreg_pf = result_logreg.select('category','prediction.result').toPandas()
result_logreg_pf['result'] = result_logreg_pf.result.apply(lambda x : x[0])

print (classification_report(result_logreg_pf['category'], result_logreg_pf['result']))

              precision    recall  f1-score   support

         neg       0.91      0.92      0.92      3317
         pos       0.80      0.79      0.79      1344

    accuracy                           0.88      4661
   macro avg       0.86      0.85      0.86      4661
weighted avg       0.88      0.88      0.88      4661



## Using More Annotators

In [37]:
document = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")
    
token = Tokenizer() \
  .setInputCols(["document"]) \
  .setOutputCol("token")
    
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")

stopwords_cleaner = StopWordsCleaner()\
      .setInputCols("normalized")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)

stemmer = Stemmer() \
    .setInputCols(["cleanTokens"]) \
    .setOutputCol("stem")

classifier_svm = DocumentMLClassifierApproach() \
    .setInputCols("stem") \
    .setLabelCol("category") \
    .setOutputCol("prediction") \
    .setClassificationModelClass("svm")\
    .setMaxIter(5)

finisher = Finisher() \
    .setInputCols(["prediction"]) \
    .setOutputCols(["predictions"]) \
    .setCleanAnnotations(True)\



pipeline = Pipeline(stages=[document, token, normalizer, stopwords_cleaner, stemmer, classifier_svm, finisher])
    
model= pipeline.fit(train_data)

In [38]:
sample_df = spark.createDataFrame([["We report two cases of pseudoporphyria caused by naproxen and oxaprozin."],
                                   ["Special attention should be paid when attempting to sample the endometrium in patients with mullerian abnormalities."]]).toDF("text")

result = model.transform(sample_df)
result.show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------+-----------+
|text                                                                                                                |predictions|
+--------------------------------------------------------------------------------------------------------------------+-----------+
|We report two cases of pseudoporphyria caused by naproxen and oxaprozin.                                            |[pos]      |
|Special attention should be paid when attempting to sample the endometrium in patients with mullerian abnormalities.|[neg]      |
+--------------------------------------------------------------------------------------------------------------------+-----------+

